# <center> CISC 867 - Deep Learning: Project 3 </center>

# Part I: Data Preparation

## Import libraries

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from keras.callbacks import EarlyStopping
from keras.layers import BatchNormalization
from keras.optimizers import adam_v2, rmsprop_v2, gradient_descent_v2
from keras.regularizers import l2
# import keras_tuner as kt
# Load the TensorBoard notebook extension
%load_ext tensorboard 
import datetime
import math

In [2]:
# load doc into memory
def load_doc(filename):
    # open the file as read only
    file = open(filename, 'r', encoding="utf8")
    # read all text
    text = file.read()
#   text = file.readline()
    # close the file
    file.close()
    return text

In [3]:
# file = open(filename, 'r', encoding="utf8")

In [4]:
# load document
# in_filename = 'The Republic by Plato.txt'
in_filename = 'Edited Republic by Plato.txt'
doc = load_doc(in_filename)
print(doc[:200])

﻿BOOK I.


I went down yesterday to the Piraeus with Glaucon the son of Ariston,
that I might offer up my prayers to the goddess (Bendis, the Thracian
Artemis.); and also because I wanted to see in wh


### Clean Text

In [5]:
import string

# turn a doc into clean tokens
def clean_doc(doc):
    # replace '--' with a space ' '
    doc = doc.replace('--', ' ')
    # split into tokens by white space
    tokens = doc.split()
    # remove punctuation from each token
    table = str.maketrans('', '', string.punctuation)
    tokens = [w.translate(table) for w in tokens]
    # remove remaining tokens that are not alphabetic
    tokens = [word for word in tokens if word.isalpha()]
    # make lower case
    tokens = [word.lower() for word in tokens]
    return tokens

In [6]:
# clean document
tokens = clean_doc(doc)
print(tokens[:200])
print('Total Tokens: %d' % len(tokens))
print('Unique Tokens: %d' % len(set(tokens)))

['i', 'i', 'went', 'down', 'yesterday', 'to', 'the', 'piraeus', 'with', 'glaucon', 'the', 'son', 'of', 'ariston', 'that', 'i', 'might', 'offer', 'up', 'my', 'prayers', 'to', 'the', 'goddess', 'bendis', 'the', 'thracian', 'artemis', 'and', 'also', 'because', 'i', 'wanted', 'to', 'see', 'in', 'what', 'manner', 'they', 'would', 'celebrate', 'the', 'festival', 'which', 'was', 'a', 'new', 'thing', 'i', 'was', 'delighted', 'with', 'the', 'procession', 'of', 'the', 'inhabitants', 'but', 'that', 'of', 'the', 'thracians', 'was', 'equally', 'if', 'not', 'more', 'beautiful', 'when', 'we', 'had', 'finished', 'our', 'prayers', 'and', 'viewed', 'the', 'spectacle', 'we', 'turned', 'in', 'the', 'direction', 'of', 'the', 'city', 'and', 'at', 'that', 'instant', 'polemarchus', 'the', 'son', 'of', 'cephalus', 'chanced', 'to', 'catch', 'sight', 'of', 'us', 'from', 'a', 'distance', 'as', 'we', 'were', 'starting', 'on', 'our', 'way', 'home', 'and', 'told', 'his', 'servant', 'to', 'run', 'and', 'bid', 'us', '

### Save Clean Text

In [7]:
# organize into sequences of tokens
length = 50 + 1
sequences = list()
for i in range(length, len(tokens)):
    # select sequence of tokens
    seq = tokens[i-length:i]
    # convert into a line
    line = ' '.join(seq)
    # store
    sequences.append(line)
print('Total Sequences: %d' % len(sequences))

Total Sequences: 117290


In [8]:
# save tokens to file, one dialog per line
def save_doc(lines, filename):
    data = '\n'.join(lines)
    file = open(filename, 'w', encoding="utf8")
    file.write(data)
    file.close()

In [9]:
# save sequences to file
out_filename = 'total_sequences.txt'
save_doc(sequences, out_filename)

## Train Language Model

### Load Sequences

In [11]:
# load doc into memory
def load_doc(filename):
    # open the file as read only
    file = open(filename, 'r', encoding="utf8")
    # read all text
    text = file.read()
    # close the file
    file.close()
    return text

# load
in_filename = 'total_sequences.txt'
doc = load_doc(in_filename)
lines = doc.split('\n')

### Divide the text into sequence of tokens

In [12]:
from tensorflow.keras.preprocessing.text import Tokenizer
# integer encode sequences of words
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)
sequences = tokenizer.texts_to_sequences(lines)

In [13]:
# vocabulary size
vocab_size = len(tokenizer.word_index) + 1

### Sequence Inputs and Output

In [14]:
# separate into input and output
sequences = np.array(sequences)
X, y = sequences[:,:-1], sequences[:,-1]
y = to_categorical(y, num_classes=vocab_size)
seq_length = X.shape[1]

MemoryError: Unable to allocate 3.20 GiB for an array with shape (117290, 7324) and data type float32

### Fit Model

### SimpleRNN Model

In [16]:
from keras.layers import Embedding, SimpleRNN, Dense

# define model
rnn_model = Sequential()
rnn_model.add(Embedding(vocab_size, 50, input_length=seq_length))
rnn_model.add(SimpleRNN(100, 
                        activation='relu', 
                        return_sequences=True))
rnn_model.add(SimpleRNN(100))
rnn_model.add(Dense(100, activation='relu'))
rnn_model.add(Dense(vocab_size, activation='softmax'))
print(rnn_model.summary())

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 50, 50)            366200    
                                                                 
 simple_rnn_2 (SimpleRNN)    (None, 50, 128)           22912     
                                                                 
 simple_rnn_3 (SimpleRNN)    (None, 128)               32896     
                                                                 
 dense_4 (Dense)             (None, 128)               16512     
                                                                 
 dense_5 (Dense)             (None, 7324)              944796    
                                                                 
Total params: 1,383,316
Trainable params: 1,383,316
Non-trainable params: 0
_________________________________________________________________
None


In [17]:
# compile model
rnn_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit model
rnn_model.fit(X, y, batch_size=128, epochs=100)

Epoch 1/100
917/917 [==============================] - 115s 124ms/step - loss: 5.9851 - accuracy: 0.0882
Epoch 2/100
917/917 [==============================] - 118s 128ms/step - loss: 5.4252 - accuracy: 0.1311
Epoch 3/100
917/917 [==============================] - 116s 126ms/step - loss: 5.1634 - accuracy: 0.1524
Epoch 4/100
917/917 [==============================] - 116s 126ms/step - loss: 4.9712 - accuracy: 0.1685
Epoch 5/100
917/917 [==============================] - 113s 123ms/step - loss: 4.8086 - accuracy: 0.1797
Epoch 6/100
917/917 [==============================] - 106s 116ms/step - loss: 4.6666 - accuracy: 0.1866
Epoch 7/100
917/917 [==============================] - 109s 119ms/step - loss: 4.5328 - accuracy: 0.1947
Epoch 8/100
917/917 [==============================] - 109s 119ms/step - loss: 4.4084 - accuracy: 0.2003
Epoch 9/100
917/917 [==============================] - 105s 114ms/step - loss: 4.2880 - accuracy: 0.2067
Epoch 10/100
917/917 [==============================] -

### LSTM Model

In [27]:
from keras.layers import LSTM

# define model
lstm_model = Sequential()
lstm_model.add(Embedding(vocab_size, 50, 
                         input_length=seq_length))
lstm_model.add(LSTM(100, 
                    return_sequences=True))
lstm_model.add(LSTM(100))
lstm_model.add(Dense(100, 
                     activation='relu'))
lstm_model.add(Dense(vocab_size, 
                     activation='softmax'))
print(lstm_model.summary())

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_6 (Embedding)     (None, 50, 50)            366200    
                                                                 
 lstm_6 (LSTM)               (None, 50, 100)           60400     
                                                                 
 lstm_7 (LSTM)               (None, 100)               80400     
                                                                 
 dense_12 (Dense)            (None, 100)               10100     
                                                                 
 dense_13 (Dense)            (None, 7324)              739724    
                                                                 
Total params: 1,256,824
Trainable params: 1,256,824
Non-trainable params: 0
_________________________________________________________________
None


In [28]:
# compile model
lstm_model.compile(
    loss='categorical_crossentropy', 
    optimizer='adam', 
    metrics=['accuracy'])

In [29]:
# fit model
history = lstm_model.fit(
    X,
    y,
    epochs=100,
    batch_size=128,
#     validation_split=0.2,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(monitor='accuracy', patience=5)
    ],
    verbose=1
)

Epoch 1/100
917/917 [==============================] - 335s 363ms/step - loss: 6.1246 - accuracy: 0.0760
Epoch 2/100
917/917 [==============================] - 188s 205ms/step - loss: 5.6511 - accuracy: 0.1091
Epoch 3/100
917/917 [==============================] - 294s 320ms/step - loss: 5.4269 - accuracy: 0.1321
Epoch 4/100
917/917 [==============================] - 336s 367ms/step - loss: 5.2795 - accuracy: 0.1447
Epoch 5/100
917/917 [==============================] - 207s 225ms/step - loss: 5.1703 - accuracy: 0.1535
Epoch 6/100
917/917 [==============================] - 222s 242ms/step - loss: 5.0789 - accuracy: 0.1590
Epoch 7/100
917/917 [==============================] - 227s 247ms/step - loss: 4.9941 - accuracy: 0.1638
Epoch 8/100
917/917 [==============================] - 238s 260ms/step - loss: 4.9149 - accuracy: 0.1691
Epoch 9/100
917/917 [==============================] - 247s 270ms/step - loss: 4.8448 - accuracy: 0.1725
Epoch 10/100
917/917 [==============================] -

### GRU Model

In [30]:
from keras.layers import GRU

# define model
gru_model = Sequential()
gru_model.add(Embedding(vocab_size, 50, input_length=seq_length))
gru_model.add(GRU(100, 
                  activation='relu', 
                  return_sequences=True))
gru_model.add(GRU(100))
gru_model.add(Dense(100, activation='relu'))
gru_model.add(Dense(vocab_size, activation='softmax'))
print(gru_model.summary())

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_7 (Embedding)     (None, 50, 50)            366200    
                                                                 
 gru (GRU)                   (None, 50, 100)           45600     
                                                                 
 gru_1 (GRU)                 (None, 100)               60600     
                                                                 
 dense_14 (Dense)            (None, 100)               10100     
                                                                 
 dense_15 (Dense)            (None, 7324)              739724    
                                                                 
Total params: 1,222,224
Trainable params: 1,222,224
Non-trainable params: 0
_________________________________________________________________
None


In [31]:
# compile model
gru_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit model
gru_model.fit(X, y, batch_size=128, epochs=100)

Epoch 1/100
917/917 [==============================] - 327s 351ms/step - loss: 6.2862 - accuracy: 0.0589
Epoch 2/100
917/917 [==============================] - 331s 361ms/step - loss: 6.1256 - accuracy: 0.0593
Epoch 3/100
917/917 [==============================] - 331s 361ms/step - loss: 6.1174 - accuracy: 0.0599
Epoch 4/100
917/917 [==============================] - 331s 361ms/step - loss: 6.1016 - accuracy: 0.0598
Epoch 5/100
917/917 [==============================] - 331s 361ms/step - loss: 6.0729 - accuracy: 0.0592
Epoch 6/100
917/917 [==============================] - 332s 363ms/step - loss: 6.0340 - accuracy: 0.0597
Epoch 7/100
917/917 [==============================] - 331s 361ms/step - loss: 5.9854 - accuracy: 0.0606
Epoch 8/100
917/917 [==============================] - 332s 362ms/step - loss: 5.9338 - accuracy: 0.0626
Epoch 9/100
917/917 [==============================] - 329s 359ms/step - loss: 5.8846 - accuracy: 0.0644
Epoch 10/100
917/917 [==============================] -

ResourceExhaustedError: Graph execution error:

Detected at node 'Adam/Adam/update/ReadVariableOp_2' defined at (most recent call last):
    File "C:\Users\osama\anaconda3\lib\runpy.py", line 194, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\Users\osama\anaconda3\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "C:\Users\osama\anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
      app.launch_new_instance()
    File "C:\Users\osama\anaconda3\lib\site-packages\traitlets\config\application.py", line 846, in launch_instance
      app.start()
    File "C:\Users\osama\anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 677, in start
      self.io_loop.start()
    File "C:\Users\osama\anaconda3\lib\site-packages\tornado\platform\asyncio.py", line 199, in start
      self.asyncio_loop.run_forever()
    File "C:\Users\osama\anaconda3\lib\asyncio\base_events.py", line 570, in run_forever
      self._